<a href="https://colab.research.google.com/github/seojeongyun/naver_movie_review_sentiment_analysis/blob/main/naver_movie_review_sentimentation_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd "/content/drive/My Drive/ratings_train.txt"

[Errno 20] Not a directory: '/content/drive/My Drive/ratings_train.txt'
/content


In [4]:
!git clone https://github.com/seojeongyun/naver_movie_review_sentiment_analysis

Cloning into 'naver_movie_review_sentiment_analysis'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 1), reused 12 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 8.06 MiB | 13.96 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [5]:
import urllib

if __name__ == '__main__':
    # Data download from url
    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                               filename="/content/drive/My Drive/ratings_train.txt")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                               filename="/content/drive/My Drive/ratings_test.txt")

In [7]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 52.2 MB/s eta 0:00:00


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import urllib
from sklearn.model_selection import train_test_split
from konlpy.tag import Mecab
from konlpy.tag import Okt

class preprocess:
    def __init__(self):
        # By using pandas, data save
        self.train_data = pd.read_table('/content/drive/My Drive/ratings_train.txt')
        self.test_data = pd.read_table('/content/drive/My Drive/ratings_test.txt')
        self.stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
        self.Okt = Okt()

    def dataset_len(self, type: str):  # Check the number of datasets
        if type == 'train':
            print('The number of train reviews : ', len(self.train_data))
        else:
            print('The number of test reviews : ', len(self.test_data))

    def process(self, type: str):
        if type == 'train':
            # Remove duplications column of document
            self.train_data.drop_duplicates(subset=['document'], inplace=True)

            # Remove the samples with null value
            train_data = self.train_data.dropna(how='any')

            # Remove special characters with regular expression
            train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
            train_data[:5]

            # Change the white space value to Null value and then remove
            train_data['document'] = train_data['document'].str.replace('^ +', "")  # change the white space to empty value
            train_data['document'].replace('', np.nan, inplace=True)
            print(train_data.isnull().sum())
            train_data = train_data.dropna(how='any')

            y_train = np.array(train_data['label']) # get labels
            return train_data, y_train

        else:
            # Apply same preprocess to test dataset
            self.test_data.drop_duplicates(subset=['document'], inplace=True)

            # Remove the samples with null value
            test_data = self.train_data.dropna(how='any')

            # remove duplicate value for column of ducument
            test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
            test_data[:5]

            test_data['document'] = self.test_data['document'].str.replace('^ +', "")  # change the white space to empty value
            test_data['document'].replace('', np.nan, inplace=True)  # change epmty space to null value
            test_data = test_data.dropna(how='any')  # remove the null value
            y_test = np.array(test_data['label'])
            return test_data, y_test

    def check_duplication(self):  # Check duplications column of document and label
        self.train_data['document'].nunique(), self.train_data['label'].nunique()
        self.test_data['document'].nunique(), self.test_data['label'].nunique()

    def remove_stopword(self, type: str, data):
        if type == 'train':
            X_train = []
            for sentence in tqdm(data['document']):
                tokenized_sentence = (self.Okt.morphs(sentence, stem=True))  # 토큰화
                stopwords_removed_sentence = [word for word in tokenized_sentence if not word in self.stopwords]  # 불용어 제거
                X_train.append(stopwords_removed_sentence)
            return X_train

        else:
            X_test = []
            for sentence in tqdm(data['document']):
                tokenized_sentence = self.Okt.morphs(sentence, stem=True)  # 토큰화
                stopwords_removed_sentence = [word for word in tokenized_sentence if not word in self.stopwords]  # 불용어 제거
                X_test.append(stopwords_removed_sentence)
            return X_test

if __name__ == '__main__':
    data = preprocess()

    data.dataset_len('train')
    data.dataset_len('test')

    data.check_duplication()

    data.process('train')
    data.process('test')

    # Check the ratio of train and labels
    data.train_data['label'].value_counts()

    # Check the null value from train set
    print(data.train_data.isnull().values.any())
    print(data.train_data.isnull().sum())
    data.train_data.loc[data.train_data.document.isnull()]

    print('The number of test dataset after preprocess :',len(data.test_data))

The number of train reviews :  150000
The number of test reviews :  50000


<ipython-input-8-9d4d4d064f29>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-8-9d4d4d064f29>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-8-9d4d4d064f29>:40: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "")  # change the white space to empty value
<ipython-input-8-9d4d4d064f29>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

id            0
document    789
label         0
dtype: int64


<ipython-input-8-9d4d4d064f29>:56: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
<ipython-input-8-9d4d4d064f29>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
<ipython-input-8-9d4d4d064f29>:59: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = self.test_data['document'].str.replace('^ +', "")  # change the white space to empty value
<ipython-input-8-9d4d4d064f29>:59: SettingWithCopyWarning: 
A value is try

True
id          0
document    1
label       0
dtype: int64
The number of test dataset after preprocess : 49158


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import re
import numpy as np
import urllib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from konlpy.tag import Mecab
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
# Tokenize
class Tokenize:
    def __init__(self):
        self.tokenizer = Tokenizer()
        self.stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
        self.threshold = 3
        self.total_cnt = len(self.tokenizer.word_index) # 단어의 수
        self.rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
        self.total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
        self.rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

    def Tokenizer(self, data):
        return self.tokenizer.fit_on_texts(data)


    def pairing(self):
        # 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
        for key, value in self.tokenizer.word_counts.items():
            self.total_freq = self.total_freq + value

            # 단어의 등장 빈도수가 threshold보다 작으면
            if(value < self.threshold):
                self.rare_cnt = self.rare_cnt + 1
                self.Tokenizerrare_freq = self.rare_freq + value

        print('단어 집합(vocabulary)의 크기 :',self.total_cnt)
        print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(self.threshold - 1, self.rare_cnt))
        print("단어 집합에서 희귀 단어의 비율:", (self.rare_cnt / self.total_cnt)*100)
        print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (self.rare_freq / self.total_freq)*100)

        ### 등장 빈도수 보다 작으면 제거
        self.vocab_size = self.total_cnt - self.rare_cnt + 1
        print('단어 집합의 크기 :',self.vocab_size)

        return self.vocab_size

        #
    def text2int(self, data):
        tokenizer = Tokenizer(self.vocab_size)
        tokenizer.fit_on_texts(data)
        if len(data) > 100000:
            X_train = tokenizer.texts_to_sequences(data)
            return X_train
        else:
            X_test = tokenizer.texts_to_sequences(data)
            return X_test

def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))


if __name__ == '__main__':
    ### 클래스 선언
    token = Tokenize()
    data = preprocess()

    ### 데이터 전처리
    X_train, Y_train = data.process('train')
    X_test, Y_test = data.process('test')

    ### 불용어 제거
    X_train = data.remove_stopword('train', X_train)
    X_test = data.remove_stopword('test', X_test)

<ipython-input-8-9d4d4d064f29>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-8-9d4d4d064f29>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-8-9d4d4d064f29>:40: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "")  # change the white space to empty value
<ipython-input-8-9d4d4d064f29>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

id            0
document    789
label         0
dtype: int64


<ipython-input-8-9d4d4d064f29>:56: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
<ipython-input-8-9d4d4d064f29>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['document'] = self.test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")  # Apply regular expression
<ipython-input-8-9d4d4d064f29>:59: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = self.test_data['document'].str.replace('^ +', "")  # change the white space to empty value
<ipython-input-8-9d4d4d064f29>:59: SettingWithCopyWarning: 
A value is try

In [13]:
if __name__ == '__main__':
# ### 토큰화 및 정수 인코딩
    # train_token = token.Tokenizer(X_train)
    # test_token = token.Tokenizer(X_test)

    ### 등장 빈도 수 낮은 단어 제외
    vocab_size = token.pairing()

    ### text2sequence
    train_token = token.text2int(X_train)
    test_token = token.text2int(X_test)

    ### remove empty sample
    drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
    X_train = np.delete(X_train, drop_train, axis=0)
    Y_train = np.delete(Y_train, drop_train, axis=0)

    ### padding
    max_len = 30
    below_threshold_len(max_len, X_train)
    X_train = pad_sequences(X_train, maxlen=max_len)
    X_test = pad_sequences(X_test, maxlen=max_len)

단어 집합(vocabulary)의 크기 : 0
등장 빈도가 2번 이하인 희귀 단어의 수: 0


ZeroDivisionError: ignored

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch

In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, embedding_dim, dropout):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=0)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=dropout)
    self.linear = nn.Linear(hidden_dim, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    embed = self.dropout(self.embedding(x))
    output, _ = self.rnn(embed)
    output = self.linear(output[:, -1, :])
    output = nn.sigmoid(output)
    return output

  def _init_state(self, batch_size=1):
    weight = next(self.parameters()).data
    return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [ ]:
embedding_dim = 100
hidden_units = 128

batch_size = 5
model = LSTM(input_dim=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
train_iter = data.Iterator.splits((X_train), batch_size=batch_size, sort=False)

In [ ]:
def train(model, train_iter):
  model.train()

  epoch_loss, epoch_acc = 0, 0
  for batch in train_iter:
    optimizer.zero_grad()
    x, y = batch.text, batch.label
    y_hat = model(x).squeeze(1)

    loss = criterion(y_hat, y)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  return epoch_loss / len(train_iter), epoch_acc / len(train_iter)

In [ ]:
for _epoch in range(1,6): # 5 epoch
  train_loss, train_acc = train(model, train_iter)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot()

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)



In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))